In [145]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [146]:
df = pd.read_csv('Rare_Event_Classification_Data.csv')
y = df['y']
df.drop(columns=['y'], inplace=True)
categorical_columns = ['x28', 'x61','time']
numerical_columns = [col for col in df.columns if col not in categorical_columns]
df_numerical = df[numerical_columns]
df_categorical = df[categorical_columns]

In [147]:
encoder = OneHotEncoder()
x28_encoded = encoder.fit_transform(df_categorical[['x28']])

x28_encoded_df = pd.DataFrame(x28_encoded.toarray(), columns=encoder.get_feature_names_out(['x28']))

df_categorical.drop(columns=['x28'], inplace=True)

df_categorical = pd.concat([x28_encoded_df, df_categorical], axis=1)

In [148]:
scaler = StandardScaler()
df_numerical_standardized = scaler.fit_transform(df_numerical)

pca = PCA(n_components=32,whiten=True,random_state=42)  
df_numerical_pca = pca.fit_transform(df_numerical_standardized)

df_pca = pd.DataFrame(data=df_numerical_pca, columns=[f'PC{i+1}' for i in range(df_numerical_pca.shape[1])])

df_final = pd.concat([df_pca, df_categorical, y], axis=1)

In [149]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [150]:
df_final['y_shifted'] = df_final['y'].shift(-1)
df_final['y_shifted'].fillna(0, inplace=True)

In [151]:
df_final['time'] = pd.to_datetime(df_final['time'],format = '%m/%d/%y %H:%M')
date = '1999-05-22'
date = datetime.strptime(date,'%Y-%m-%d')
X_train = df_final[df_final['time'] < date]
X_test = df_final[df_final['time'] >= date]
y_train = X_train['y_shifted']
X_train.drop(columns=['y','time','y_shifted'], inplace=True)
y_test = X_test['y_shifted']
X_test.drop(columns=['y','time','y_shifted'],inplace=True)


In [152]:
svm_classifier = SVC(C=1.8, kernel='rbf', class_weight='balanced',gamma=0.007)

svm_classifier.fit(X_train, y_train)

y_pred = svm_classifier.predict(X_test)

In [153]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.8845429826396152
Precision: 0.020522388059701493
Recall: 0.2894736842105263
F1 Score: 0.03832752613240418
